# Content Based Rec Sys

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [3]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
users = pd.read_csv('users.csv')

### Item-Item Similarity Baserd Rec

In [4]:
movies.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [15]:
select_movies = ratings.movieId.value_counts().head(1000).index.to_list()
movies = movies.loc[movies.movieId.isin(select_movies)]
ratings = ratings.loc[ratings.movieId.isin(select_movies)]

In [23]:
m = movies.copy()
m['genres'] = m['genres'].str.split('|')
m = m.explode('genres')
m = m.pivot(index='movieId', columns='genres', values='title')
m = ~m.isna()
m = m.astype(int)

In [35]:
m.shape

(1000, 19)

In [24]:
m.head()

genres,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,
1,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0


In [36]:
a = m.iloc[0].values
b = m.iloc[1].values

In [37]:
a

array([0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [38]:
b

array([0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [42]:
def hamming_distance(x, y):
    return sum(abs(x-y))

In [43]:
query

1

In [46]:
ranks = []

for query in m.index[:10]:
    for candidate in m.index:
        if candidate == query:
            continue
        ranks.append([query, candidate, hamming_distance(m.loc[query], m.loc[candidate])])

In [48]:
ranks = pd.DataFrame(ranks, columns=['query', 'candidate', 'distance'])
ranks = ranks.merge(movies[['movieId', 'title']], left_on='query', right_on='movieId').rename(columns={'title': 'query_tittle'}).drop(columns=['movieId'])
ranks = ranks.merge(movies[['movieId', 'title']], left_on='candidate', right_on='movieId').rename(columns={'title': 'candidate_tittle'}).drop(columns=['movieId'])
ranks = ranks.sort_values(by=['query', 'distance'])
ranks.head()

,query,candidate,distance,query_tittle,candidate_tittle
5391,1,2294,0,Toy Story (1995),Antz (1998)
6651,1,3114,0,Toy Story (1995),Toy Story 2 (1999)
7921,1,4886,0,Toy Story (1995),"Monsters, Inc. (2001)"
1871,1,673,1,Toy Story (1995),Space Jam (1996)
5491,1,2355,1,Toy Story (1995),"Bug's Life, A (1998)"


In [49]:
ranks.query_tittle.unique()

array(['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)',
       'Father of the Bride Part II (1995)', 'Heat (1995)',
       'Sabrina (1995)', 'GoldenEye (1995)',
       'American President, The (1995)', 'Casino (1995)',
       'Sense and Sensibility (1995)'], dtype=object)

In [51]:
ranks.loc[ranks.query_tittle == 'Casino (1995)'].head(5)

,query,candidate,distance,query_tittle,candidate_tittle
179,16,36,0,Casino (1995),Dead Man Walking (1995)
729,16,247,0,Casino (1995),Heavenly Creatures (1994)
969,16,318,0,Casino (1995),"Shawshank Redemption, The (1994)"
1299,16,431,0,Casino (1995),Carlito's Way (1993)
2159,16,858,0,Casino (1995),"Godfather, The (1972)"


---
### User-User Similarity Based Rec Sys

In [7]:
users.head(2)

,userId,age,time_spent_per_day
0,1,16,3.976315
1,2,24,1.891303


In [27]:
r = ratings.copy()

In [29]:
r['hour'] = r['timestamp'].apply(lambda x: datetime.fromtimestamp(x).hour)
r.head()

,userId,movieId,rating,timestamp,hour
0,1,16,4.0,1217897793,6
1,1,24,1.5,1217895807,5
2,1,32,4.0,1217896246,6
3,1,47,4.0,1217896556,6
4,1,50,4.0,1217896523,6


In [33]:
r.groupby('userId').rating.mean().reset_index().head()

,userId,rating
0,1,3.691589
1,2,3.923077
2,3,3.806452
3,4,4.159420
4,5,2.864865


In [30]:
users = users.merge(r.groupby('userId').rating.mean().reset_index(), on='userId')
users = users.merge(r.groupby('userId').hour.mean().reset_index(), on='userId')

In [32]:
users.head()

,userId,age,time_spent_per_day,rating,hour
0,1,16,3.976315,3.691589,5.616822
1,2,24,1.891303,3.923077,21.000000
2,3,20,4.521478,3.806452,14.370968
3,4,23,2.095284,4.159420,8.000000
4,5,35,1.759860,2.864865,0.513514


In [53]:
u = users.copy()
u = u.set_index('userId')
u.columns = ['age', 'time_spent_per_day', 'u_avg_rating', 'hour']

In [56]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
u = pd.DataFrame(scaler.fit_transform(u), columns=u.columns, index=u.index)

In [58]:
def euclidian_distance(x, y):
    return np.linalg.norm(x-y)

In [59]:
userid = 5

In [61]:
dist = []
for user in u.index:
    dist.append(euclidian_distance(u.loc[userid], u.loc[user]))

u_rank = pd.DataFrame()
u_rank['id'] = u.index
u_rank['dist'] = dist
u_rank = u_rank.loc[u_rank.id != userid]
u_rank = u_rank.sort_values(by='dist')
u_rank.head()

,id,dist
213,214,1.400996
124,125,1.559669
301,302,1.641682
409,410,1.657114
25,26,1.676895


---

### Regression based Rec Sys

In [64]:
m.head(2)

genres,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,
1,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [65]:
u.head(2)

,age,time_spent_per_day,u_avg_rating,hour
userId,,,,
1,-1.470292,0.341073,-0.073572,-0.882006
2,-0.135616,-1.079947,0.426461,1.477906


In [68]:
X = ratings[['movieId', 'userId', 'rating']].copy()  # base df 
X = X.merge(u.reset_index(), on='userId', how='right')
X = X.merge(m.reset_index(), on='movieId', how='right')
X.head()

,movieId,userId,rating,age,time_spent_per_day,u_avg_rating,hour,Action,Adventure,Animation,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,2,5.0,-0.135616,-1.079947,0.426461,1.477906,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,5,4.0,1.699565,-1.169532,-1.859363,-1.664898,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,8,5.0,0.364888,0.298545,0.160605,1.324497,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,11,4.0,-1.303458,0.513712,-0.380602,0.557454,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,14,4.0,-0.302450,1.251552,-0.379415,0.557454,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [70]:
X = X.drop(columns = ['movieId', 'userId'])
y = X.pop('rating')

In [71]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [80]:
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [81]:
from sklearn.metrics import mean_squared_error as mse
mse(y_test, y_pred)**0.5

0.9485178995045113

---

# Collaborative Rec Sys - Matrix Factorisation

- collective matrix factorisation for recommender systems
- https://cmfrec.readthedocs.io/en/latest/

In [82]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [84]:
rm = ratings.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
rm.astype(int).head()

movieId,1,2,3,4,5,6,7,8,9,10,...,144482,144656,144976,146344,146656,146684,146878,148238,148626,149532
userId,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,0,2,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,3,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
# ! pip install cmfrec? - surprise (post read)

In [91]:
rm_raw = ratings[['userId', 'movieId', 'rating']].copy()
rm_raw.columns = ['UserId', 'ItemId', 'Rating']  # Lib requires specific column names
rm_raw.head(2)

,UserId,ItemId,Rating
0,1,16,4.0
1,1,24,1.5


In [100]:
from cmfrec import CMF

model = CMF(k=3, lambda_=0.1, user_bias=False, item_bias=False, verbose=False)
model.fit(rm_raw)

Collective matrix factorization model
(explicit-feedback variant)


In [101]:
model.A_.shape

(668, 3)

In [102]:
model.B_.shape

(10325, 3)

In [103]:
model.A_

array([[-1.0550879 ,  0.04121023,  0.77834725],
       [ 0.51935625,  0.4009357 ,  0.84999347],
       [-0.23952661,  0.3721298 ,  0.59112626],
       ...,
       [-1.0284921 ,  0.24378349,  0.62923706],
       [-0.378822  ,  0.53025264,  0.26887265],
       [-0.8036842 , -0.9499605 ,  0.7772622 ]], dtype=float32)

In [106]:
top_items = model.topN(user=4, n=10)
movies.loc[movies.movieId.isin(top_items)]

,movieId,title,genres
882,1085,"Old Man and the Sea, The (1958)",Adventure|Drama
1772,2237,Without Limits (1998),Drama
2339,2927,Brief Encounter (1946),Drama|Romance
2448,3061,Holiday Inn (1942),Comedy|Musical
2886,3655,Blow-Out (La grande bouffe) (1973),Drama
3026,3830,Psycho Beach Party (2000),Comedy|Horror|Thriller
3996,5147,Wild Strawberries (Smultronstället) (1957),Drama
5075,6987,"Cabinet of Dr. Caligari, The (Cabinet des Dr. ...",Crime|Fantasy|Horror
5211,7158,House of Sand and Fog (2003),Drama
5560,8131,Pursuit of Happiness (2001),Comedy|Romance


further read: https://nbviewer.org/github/david-cortes/cmfrec/blob/master/example/cmfrec_movielens_sideinfo.ipynb

In [107]:
rm__ = np.dot(model.A_, model.B_.T) + model.glob_mean_
mse(rm.values[rm > 0], rm__[rm > 0])**0.5

1.2845920523985803